In [46]:
import os
import pickle
from utils.b2 import B2
from utils.modeling import *
import streamlit as st
from dotenv import load_dotenv
import numpy as np
# from implicit.als import AlternatingLeastSquares
# from sklearn.metrics import mean_squared_error
# # from scipy.sparse import coo_matrix
# # import category_encoders as ce
# import pandas as pd
#import pymc as pm
# from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, balanced_accuracy_score
# from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

In [47]:
# ------------------------------------------------------
#                      APP CONSTANTS
# ------------------------------------------------------
REMOTE_DATA = 'pbp.csv'

In [48]:
# ------------------------------------------------------
#                        CONFIG
# ------------------------------------------------------
load_dotenv()

True

In [49]:
# load Backblaze connection
b2 = B2(endpoint=os.environ['B2_ENDPOINT'],
        key_id=os.environ['B2_KEYID'],
        secret_key=os.environ['B2_APPKEY'])

In [50]:
# ------------------------------------------------------
#                        CACHING
# ------------------------------------------------------
@st.cache_data
def get_data():
    # collect data frame of reviews and their sentiment
    b2.set_bucket(os.environ['B2_BUCKETNAME'])
    df_pbp = b2.get_df(REMOTE_DATA)

    return df_pbp

2024-04-10 19:04:30.656 No runtime found, using MemoryCacheStorageManager


In [51]:
data = get_data()

In [52]:
@st.cache_data
def prep_data(data):
    rush_data = data[data['IsRush'] == 1]
    pass_data = data[data['IsPass'] == 1]
    # Train pass play model
    X_pass = pass_data[['SitID', 'PlayID']]
    y_pass = pass_data['Yards']
    # Train rush play model
    X_rush = rush_data[['SitID', 'PlayID']]
    y_rush = rush_data['Yards']
    return X_pass, y_pass, X_rush, y_rush, rush_data, pass_data

2024-04-10 19:04:33.309 No runtime found, using MemoryCacheStorageManager


In [53]:
X_pass, y_pass, X_rush, y_rush, rush_data, pass_data = prep_data(data)

In [54]:
@st.cache_data
def get_model(X_pass, y_pass, X_rush, y_rush, rush_data, pass_data):
    pass_regressor = RandomForestRegressor(n_estimators=100, random_state=42)
    pass_regressor.fit(X_pass, y_pass)
    rush_regressor = RandomForestRegressor(n_estimators=100, random_state=42)
    rush_regressor.fit(X_rush, y_rush)
    rush_data.loc[:, 'Predicted Yards'] = rush_regressor.predict(X_rush)
    best_rush_plays = rush_data.sort_values(by='Predicted Yards', ascending=False)
    pass_data.loc[:, 'Predicted Yards'] = pass_regressor.predict(X_pass)
    best_pass_plays = pass_data.sort_values(by='Predicted Yards', ascending=False)
    return best_rush_plays, best_pass_plays

2024-04-10 19:04:33.455 No runtime found, using MemoryCacheStorageManager


In [55]:
best_rush_plays, best_pass_plays = get_model(X_pass, y_pass, X_rush, y_rush, rush_data, pass_data)

C:\Users\Minh\AppData\Local\Temp\ipykernel_12576\2060931429.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rush_data.loc[:, 'Predicted Yards'] = rush_regressor.predict(X_rush)
C:\Users\Minh\AppData\Local\Temp\ipykernel_12576\2060931429.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pass_data.loc[:, 'Predicted Yards'] = pass_regressor.predict(X_pass)


In [83]:
@st.cache_data
def getPlay(team, down, distance, yardline):
    teamId = data.loc[data['OffenseTeam'] == team, 'TeamID'].values[0]
    SitId = yardline + 100 * distance + 10000 * down + 100000 * teamId
    if SitId in best_pass_plays['SitID'].values:
        best_passes = best_pass_plays[best_pass_plays['SitID'] == SitId]
        best_passes = best_passes.nlargest(2, 'Predicted Yards')
        print('First Pass Choice: ', best_passes.iloc[0]['PassType'])
        print('Predicted Gain: ', round(float(best_passes.iloc[0]['Predicted Yards'])))
        print('Second Pass Choice: ', best_passes.iloc[1]['PassType'])
        print('Predicted Gain: ', round(float(best_passes.iloc[1]['Predicted Yards'])))

    else:
        best_passes = best_pass_plays[best_pass_plays['TeamID'] == teamId]
        average_yards = best_passes['Yards'].mean()
        # Select two plays nearest to the average
        nearest_plays = best_pass_plays.iloc[(best_pass_plays['Predicted Yards'] - average_yards).abs().argsort()[:2]]
        print('First Pass Choice: ', nearest_plays.iloc[0]['PassType'])
        print('Predicted Gain: ', round(float(nearest_plays.iloc[0]['Predicted Yards'])))
        print('Second Pass Choice: ', nearest_plays.iloc[1]['PassType'])
        print('Predicted Gain: ', round(float(nearest_plays.iloc[1]['Predicted Yards'])))


    if SitId in best_rush_plays['SitID'].values:
        best_rushes = best_rush_plays[best_rush_plays['SitID'] == SitId]
        best_rushes = best_rushes.nlargest(2, 'Predicted Yards')
        print('First Rush Choice: ', best_rushes.iloc[0]['RushDirection'])
        print('Predicted Gain: ', round(float(best_rushes.iloc[0]['Predicted Yards'])))
        print('Second Rush Choice: ', best_rushes.iloc[1]['RushDirection'])
        print('Predicted Gain: ', round(float(best_rushes.iloc[1]['Predicted Yards'])))

    else:
        best_rushes = best_rush_plays[best_rush_plays['TeamID'] == teamId]
        average_yards = best_rushes['Yards'].mean()
        # Select two plays nearest to the average
        nearest_plays = best_rush_plays.iloc[(best_rush_plays['Predicted Yards'] - average_yards).abs().argsort()[:2]]
        print('First Pass Choice: ', nearest_plays.iloc[0]['RushDirection'])
        print('Predicted Gain: ', round(float(nearest_plays.iloc[0]['Predicted Yards'])))
        print('Second Pass Choice: ', nearest_plays.iloc[1]['RushDirection'])
        print('Predicted Gain: ', round(float(nearest_plays.iloc[1]['Predicted Yards'])))

2024-04-10 20:31:21.410 No runtime found, using MemoryCacheStorageManager


In [85]:
getPlay('NE', 1, 5, 95)

First Pass Choice:  SHORT MIDDLE
Predicted Gain:  6


IndexError: single positional indexer is out-of-bounds